<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

## import

In [40]:
# Dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

# Imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler

#random
import random

# LightGBM
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# LogisticRegression
from sklearn.linear_model import LogisticRegression

# RandomForest
from sklearn.ensemble import RandomForestClassifier

In [41]:
#
#!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
#import pandas_profiling
import seaborn as sns
#from pandas_profiling.utils.cache import cache_file

#Optuna
!pip install optuna
import optuna
from sklearn.metrics import log_loss

#何で必要なのか分かってない
%matplotlib inline
warnings.filterwarnings('ignore')

## upload

In [42]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data  submit_sample.csv	test.csv  train.csv


In [43]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

## Feature Engineering

In [44]:
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#EDA:整理
data["income"]=data["job"].copy()
data["goodincome"]=data["job"].copy()
data["job"].replace(['blue-collar','management','technician','admin.','services',\
                     'unknown','self-employed','entrepreneur','student','retired',\
                     'unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["married"]=data["marital"]
data["single"]=data["marital"]
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,1,0,2],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,1,2,0],inplace=True)
data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],\
                      [3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)

#EDA:追加
data["year"]=data["month"]*30+data["day"]
data["campaign+previous"]=data["campaign"] + data["previous"]
data["poutcome/pdays"]=data["poutcome"] / (data["pdays"]+10) * 10000
data["poutcome/pdays"] = data["poutcome/pdays"].astype(np.int64)
data["income"].replace(['blue-collar','management','technician',\
                                         'admin.','services','unknown','self-employed',\
                                         'entrepreneur','student','retired','unemployed','housemaid'],\
                                        [0,0,0,0,0,0,0,0,1,1,1,0],inplace=True)
data["goodincome"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],\
                                                [0,1,0,1,0,0,1,1,0,0,0,0],inplace=True)
data["housing+loan"] = data["housing"] + data["loan"]
data["duration*campaign"] = data["duration"] * data["campaign"] 
data["married"].replace(['married','single','divorced'],[1,0,1],inplace=True)
data["single"].replace(['married','single','divorced'],[1,0,0],inplace=True)
data["retirement"] = (data["age"] > 59).astype(int)

#int化
train["y"]=train["y"].astype(np.int64)

#削除
delete_columns = ['id', 'default', 'marital']
data=data.drop(delete_columns, axis=1)

#カテゴリ化
categorical_features = ['job','education','housing','loan','contact','poutcome',
                        "income",'goodincome','married',"single",'housing+loan',
                        "retirement"]
data[categorical_features] = data[categorical_features].astype('category')

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]

#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y = train['y']
X = train.drop('y', axis = 1)
X_sub = test.drop('y', axis = 1)



# LightGBM

## def LGB

In [47]:

random.randint(1,100)

63

In [48]:
def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

## Optuna of LGB

In [49]:
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.15,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }
    #LightGBMのデータ分割
    X_train_LGB, X_test_LGB, y_train_LGB, y_test_LGB = train_test_split(X, y, test_size=0.2, random_state=random.randint(1,100))
    X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X, y, test_size=0.2, random_state=random.randint(1,100))

    lgb_train = lgb.Dataset(X_train_LGB, y_train_LGB)
    lgb_eval = lgb.Dataset(X_valid_LGB, y_valid_LGB, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid_LGB = model.predict(X_valid_LGB, num_iteration=model.best_iteration)
    score = log_loss(y_valid_LGB, y_pred_valid_LGB)
    return score

In [50]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=30)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.17891	valid_1's binary_logloss: 0.182985
[20]	training's binary_logloss: 0.149551	valid_1's binary_logloss: 0.16025
[30]	training's binary_logloss: 0.128362	valid_1's binary_logloss: 0.143995
[40]	training's binary_logloss: 0.112964	valid_1's binary_logloss: 0.13371
[50]	training's binary_logloss: 0.100252	valid_1's binary_logloss: 0.124659
[60]	training's binary_logloss: 0.0891741	valid_1's binary_logloss: 0.116799
[70]	training's binary_logloss: 0.0794186	valid_1's binary_logloss: 0.10956
[80]	training's binary_logloss: 0.071558	valid_1's binary_logloss: 0.104496
[90]	training's binary_logloss: 0.0644205	valid_1's binary_logloss: 0.0990576
[100]	training's binary_logloss: 0.0588354	valid_1's binary_logloss: 0.0955216
[110]	training's binary_logloss: 0.052721	valid_1's binary_logloss: 0.0915913
[120]	training's binary_logloss: 0.0474657	valid_1's binary_logloss: 0.0882825
[130]	training's b

[I 2020-08-29 02:33:18,994] Trial 0 finished with value: 0.07114871296187461 and parameters: {'max_bin': 427, 'num_leaves': 79}. Best is trial 0 with value: 0.07114871296187461.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.169625	valid_1's binary_logloss: 0.176391
[20]	training's binary_logloss: 0.13791	valid_1's binary_logloss: 0.149832
[30]	training's binary_logloss: 0.114696	valid_1's binary_logloss: 0.130627
[40]	training's binary_logloss: 0.0970713	valid_1's binary_logloss: 0.117322
[50]	training's binary_logloss: 0.0834955	valid_1's binary_logloss: 0.106789
[60]	training's binary_logloss: 0.0730605	valid_1's binary_logloss: 0.0990287
[70]	training's binary_logloss: 0.0643309	valid_1's binary_logloss: 0.09317
[80]	training's binary_logloss: 0.0566389	valid_1's binary_logloss: 0.0872514
[90]	training's binary_logloss: 0.0500922	valid_1's binary_logloss: 0.08262
[100]	training's binary_logloss: 0.0450456	valid_1's binary_logloss: 0.0791196
[110]	training's binary_logloss: 0.0393772	valid_1's binary_logloss: 0.0756254
[120]	training's binary_logloss: 0.0354313	valid_1's binary_logloss: 0.0737645
[130]	traini

[I 2020-08-29 02:33:21,504] Trial 1 finished with value: 0.06221087381073065 and parameters: {'max_bin': 372, 'num_leaves': 96}. Best is trial 1 with value: 0.06221087381073065.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.17321	valid_1's binary_logloss: 0.186315
[20]	training's binary_logloss: 0.139391	valid_1's binary_logloss: 0.157593
[30]	training's binary_logloss: 0.115384	valid_1's binary_logloss: 0.136225
[40]	training's binary_logloss: 0.0972407	valid_1's binary_logloss: 0.120025
[50]	training's binary_logloss: 0.0849238	valid_1's binary_logloss: 0.11012
[60]	training's binary_logloss: 0.0735808	valid_1's binary_logloss: 0.100834
[70]	training's binary_logloss: 0.0643039	valid_1's binary_logloss: 0.093828
[80]	training's binary_logloss: 0.0564671	valid_1's binary_logloss: 0.0875609
[90]	training's binary_logloss: 0.0497113	valid_1's binary_logloss: 0.0821393
[100]	training's binary_logloss: 0.0439522	valid_1's binary_logloss: 0.077695
[110]	training's binary_logloss: 0.0388076	valid_1's binary_logloss: 0.0738451
[120]	training's binary_logloss: 0.034102	valid_1's binary_logloss: 0.0704288
[130]	trainin

...

[I 2020-08-29 02:34:41,842] Trial 27 finished with value: 0.06995327133473958 and parameters: {'max_bin': 499, 'num_leaves': 55}. Best is trial 11 with value: 0.05723119627363833.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.170619	valid_1's binary_logloss: 0.182714
[20]	training's binary_logloss: 0.136929	valid_1's binary_logloss: 0.156492
[30]	training's binary_logloss: 0.111927	valid_1's binary_logloss: 0.13614
[40]	training's binary_logloss: 0.0943821	valid_1's binary_logloss: 0.121934
[50]	training's binary_logloss: 0.0796841	valid_1's binary_logloss: 0.109991
[60]	training's binary_logloss: 0.0686807	valid_1's binary_logloss: 0.100441
[70]	training's binary_logloss: 0.0600347	valid_1's binary_logloss: 0.0932899
[80]	training's binary_logloss: 0.0527922	valid_1's binary_logloss: 0.0879424
[90]	training's binary_logloss: 0.0467907	valid_1's binary_logloss: 0.0830446
[100]	training's binary_logloss: 0.0405027	valid_1's binary_logloss: 0.0785544
[110]	training's binary_logloss: 0.0359533	valid_1's binary_logloss: 0.0749069
[120]	training's binary_logloss: 0.0314455	valid_1's binary_logloss: 0.0714828
[130]	tra

[I 2020-08-29 02:34:44,480] Trial 28 finished with value: 0.06054910486411089 and parameters: {'max_bin': 418, 'num_leaves': 107}. Best is trial 11 with value: 0.05723119627363833.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.177322	valid_1's binary_logloss: 0.180527
[20]	training's binary_logloss: 0.145843	valid_1's binary_logloss: 0.15556
[30]	training's binary_logloss: 0.122909	valid_1's binary_logloss: 0.137987
[40]	training's binary_logloss: 0.105759	valid_1's binary_logloss: 0.124263
[50]	training's binary_logloss: 0.0927721	valid_1's binary_logloss: 0.114639
[60]	training's binary_logloss: 0.0820242	valid_1's binary_logloss: 0.1071
[70]	training's binary_logloss: 0.0732208	valid_1's binary_logloss: 0.100775
[80]	training's binary_logloss: 0.0648668	valid_1's binary_logloss: 0.0953509
[90]	training's binary_logloss: 0.0575055	valid_1's binary_logloss: 0.0897094
[100]	training's binary_logloss: 0.0517957	valid_1's binary_logloss: 0.0862376
[110]	training's binary_logloss: 0.0467776	valid_1's binary_logloss: 0.0827952
[120]	training's binary_logloss: 0.0427543	valid_1's binary_logloss: 0.0804151
[130]	trainin

[I 2020-08-29 02:34:47,142] Trial 29 finished with value: 0.0647395536255776 and parameters: {'max_bin': 369, 'num_leaves': 87}. Best is trial 11 with value: 0.05723119627363833.


In [51]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': study.best_params["num_leaves"],
    'boosting_type' : 'gbdt',
    'objective': 'binary',
    'max_bin': study.best_params["max_bin"],
    'max_dapth': 4
}



##Bagging of LGB

In [52]:
def bagging(seed):
    positive_count_train = y_train_LGB.sum()
    sampler = RandomUnderSampler(sampling_strategy={0:int(positive_count_train), 1:int(positive_count_train)},random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train_LGB, y_train_LGB)
    X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=seed)
    model_bagging = lgbm_train(X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB, lgbm_params)
    return model_bagging

In [53]:
%%time
models_LGB = []
for i in range(10):
    models_LGB.append(bagging(i))



[1]	valid_0's binary_logloss: 0.682224
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.666386
[3]	valid_0's binary_logloss: 0.652149
[4]	valid_0's binary_logloss: 0.639149
[5]	valid_0's binary_logloss: 0.627409
[6]	valid_0's binary_logloss: 0.616075
[7]	valid_0's binary_logloss: 0.606826
[8]	valid_0's binary_logloss: 0.597958
[9]	valid_0's binary_logloss: 0.588876
[10]	valid_0's binary_logloss: 0.581227
[11]	valid_0's binary_logloss: 0.573775
[12]	valid_0's binary_logloss: 0.565043
[13]	valid_0's binary_logloss: 0.558888
[14]	valid_0's binary_logloss: 0.552048
[15]	valid_0's binary_logloss: 0.546129
[16]	valid_0's binary_logloss: 0.539375
[17]	valid_0's binary_logloss: 0.534497
[18]	valid_0's binary_logloss: 0.529654
[19]	valid_0's binary_logloss: 0.525158
[20]	valid_0's binary_logloss: 0.519672
[21]	valid_0's binary_logloss: 0.516266
[22]	valid_0's binary_logloss: 0.512504
[23]	valid_0's binary_logloss: 0.508755
[24]	valid_0's binary_loglo

In [54]:
y_preds_LGB = []

for m in models_LGB:
    y_preds_LGB.append(m.predict(X_test_LGB, num_iteration=m.best_iteration))

y_preds_bagging_LGB = sum(y_preds_LGB)/len(y_preds_LGB)
# auc を計算する
auc = roc_auc_score(y_test_LGB, y_preds_bagging_LGB)
print(auc)

y_subs_LGB = []
for m in models_LGB:
    y_subs_LGB.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging_LGB = sum(y_subs_LGB)/len(y_subs_LGB)
# auc を計算する
y_subs_bagging_LGB

0.8412451081518109


array([0.97792507, 0.36424535, 0.11592878, ..., 0.34538758, 0.0099171 ,
       0.7067542 ])

In [55]:
percentage_LGB=0.23
y_sub_LGB = (y_subs_bagging_LGB > percentage_LGB).astype(int)

# sub['B'] = y_sub_LGB
# sub.to_csv('submission_lightgbm_bagging.csv', index=False, header=False)

sum(y_sub_LGB),len(y_sub_LGB)

(8719, 18050)

# Output

In [ ]:
sub['B'] = y_sub_LGB
sub.to_csv('submission_LGB.csv', index=False, header=False)
sub.head()

,A,B
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


In [64]:
files.download('submission_LGB.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>